<a href="https://colab.research.google.com/github/dhruthick/cse256project/blob/main/data/interactions_data_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

## Read data

In [ ]:
data_path='/content/drive/MyDrive/cse256/project/data/'
# interactions=pd.read_csv(data_path+'interactions_with_trackslyrics.csv')

In [ ]:
tracks=pd.read_csv(data_path+'tracks.csv').drop('tid',axis=1)
interactions=interactions.join(tracks.set_index('track_uri'),on='track_uri',how='inner').reset_index(drop=True)


## Generate playlist ids and track ids

In [ ]:
pids=[]
for pid, group in interactions.groupby('pid'):
  if len(group)>5:
    pids.append(pid)
interactions=interactions[interactions.pid.isin(pids)]

In [ ]:
track_uri2tid={}
tid=0
for uri in np.unique(interactions.track_uri):
  track_uri2tid[uri]=tid
  tid+=1
interactions['tid']=interactions.track_uri.apply(lambda x:track_uri2tid[x])

In [ ]:
pid2gen_pid={}
gen_pid=0
for pid in np.unique(interactions.pid):
  pid2gen_pid[pid]=gen_pid
  gen_pid+=1
interactions['gen_pid']=interactions.pid.apply(lambda x:pid2gen_pid[x])

## Split interactions into train, val, and test sets

In [ ]:
train,test=train_test_split(interactions, test_size=0.4, stratify=interactions['pid'])
val,test=train_test_split(test, test_size=0.5, stratify=test['pid'])

In [ ]:
l=[len(train),len(val),len(test)]
l

[598734, 199578, 199578]

In [ ]:
train.to_csv(data_path+'interactions_train.csv',index=False)
val.to_csv(data_path+'interactions_val.csv',index=False)
test.to_csv(data_path+'interactions_test.csv',index=False)

In [ ]:
 interactions.to_csv(data_path+'all_interactions.csv',index=False)